# QG Omega Equation Solver

Demonstrates the Hoskins (1978) Q-vector QG omega equation solver in `pvtend`.

The solver uses:
- FFT in longitude (periodic BC)
- Thomas tridiagonal solver in pressure
- Latitude taper for QG validity (15°–80°N)

All data is **synthetic** — no ERA5 files needed.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pvtend.omega import solve_qg_omega, compute_geostrophic_wind, lat_taper
from pvtend.constants import R_EARTH, OMEGA_E, R_DRY, SIGMA0_CONST

## 1. Define Grid

NH grid at 2° resolution with 9 standard pressure levels (1000–100 hPa).

In [ ]:
lat = np.arange(0.0, 90.1, 2.0)   # 0°N to 90°N
lon = np.arange(-180.0, 180.0, 2.0)
plevs_pa = np.array([1000, 850, 700, 500, 400, 300, 250, 200, 100], dtype=float) * 100  # Pa

nlat, nlon, nlev = len(lat), len(lon), len(plevs_pa)
lon2d, lat2d = np.meshgrid(lon, lat)

print(f"Grid: {nlat} lat × {nlon} lon × {nlev} levels")
print(f"Pressure: {plevs_pa[0]/100:.0f}–{plevs_pa[-1]/100:.0f} hPa")

## 2. Synthetic Geopotential & Temperature

Create a baroclinic jet with a short-wave trough embedded at 40°N.

In [ ]:
# Basic temperature: decreasing with height, equator-to-pole gradient
T0 = 288.0  # surface reference
lapse = 6.5e-3  # K/m
H = 7000.0  # scale height

t_3d = np.zeros((nlev, nlat, nlon))
for k, p in enumerate(plevs_pa):
    z_approx = -H * np.log(p / plevs_pa[0])  # approximate height
    t_base = T0 - lapse * z_approx
    # Meridional gradient: ~30 K equator-to-pole
    t_meridional = -30.0 * np.sin(np.deg2rad(lat2d))**2
    # Short-wave trough at 40°N
    trough_amp = 5.0 * (p / plevs_pa[0]) * np.exp(-((lat2d - 40)**2) / 200)
    t_wave = trough_amp * np.sin(np.deg2rad(lon2d) * 5)  # wavenumber 5
    t_3d[k] = t_base + t_meridional + t_wave

print(f"Temperature range: {t_3d.min():.1f} – {t_3d.max():.1f} K")

In [ ]:
# Build geopotential by integrating the hypsometric equation
phi_3d = np.zeros_like(t_3d)
phi_3d[0] = 9.81 * 100.0  # ~100 m surface geopotential

for k in range(1, nlev):
    dp = plevs_pa[k-1] - plevs_pa[k]
    T_layer = 0.5 * (t_3d[k] + t_3d[k-1])
    phi_3d[k] = phi_3d[k-1] + R_DRY * T_layer * np.log(plevs_pa[k-1] / plevs_pa[k])

print(f"500 hPa Φ range: {phi_3d[3].min()/9.81:.0f} – {phi_3d[3].max()/9.81:.0f} gpm")

## 3. Geostrophic Wind

Compute $u_g$ and $v_g$ from the geopotential field.

In [ ]:
ug, vg = compute_geostrophic_wind(phi_3d, lat, lon, sigma_smooth=1.5)

print(f"u_g at 300 hPa: {ug[5].min():.1f} to {ug[5].max():.1f} m/s")
print(f"v_g at 300 hPa: {vg[5].min():.1f} to {vg[5].max():.1f} m/s")

In [ ]:
# Plot 300 hPa geostrophic wind
k300 = 5  # index for 300 hPa
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

im0 = axes[0].contourf(lon, lat, ug[k300], levels=20, cmap="RdBu_r")
axes[0].set_title(r"$u_g$ at 300 hPa [m/s]")
plt.colorbar(im0, ax=axes[0])

skip = 5
spd = np.sqrt(ug[k300]**2 + vg[k300]**2)
im1 = axes[1].contourf(lon, lat, spd, levels=20, cmap="YlOrRd")
axes[1].quiver(lon[::skip], lat[::skip], ug[k300, ::skip, ::skip],
               vg[k300, ::skip, ::skip], scale=300, alpha=0.6)
axes[1].set_title("Wind speed + vectors at 300 hPa")
plt.colorbar(im1, ax=axes[1], label="m/s")

for ax in axes:
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
plt.tight_layout()
plt.show()

## 4. Latitude Taper

The QG equation is invalid near the equator and singular near the pole.
A combined taper smoothly zeroes the forcing outside 15°–80°N.

In [ ]:
taper = lat_taper(lat)

fig, ax = plt.subplots(figsize=(8, 3))
ax.plot(lat, taper, 'b-', lw=2)
ax.axvspan(0, 15, alpha=0.1, color='red', label='QG invalid')
ax.axvspan(80, 90, alpha=0.1, color='red')
ax.axvspan(15, 25, alpha=0.1, color='orange', label='Taper zone')
ax.set_xlabel('Latitude (°N)')
ax.set_ylabel('Taper weight')
ax.set_title('QG Latitude Taper')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Solve QG Omega

Solve: $\nabla_p^2 \omega + \frac{f_0^2}{\sigma} \frac{\partial^2 \omega}{\partial p^2} = -2 \nabla \cdot \mathbf{Q} - \beta \frac{R_d}{\sigma p} \frac{\partial T}{\partial x}$

In [ ]:
omega_dry = solve_qg_omega(
    ug, vg, t_3d, lat, lon, plevs_pa,
    use_constant_sigma=True,
    sigma0_const=SIGMA0_CONST,
)

print(f"ω shape: {omega_dry.shape}")
print(f"ω range at 500 hPa: {omega_dry[3].min():.4f} to {omega_dry[3].max():.4f} Pa/s")
print(f"                   ({omega_dry[3].min()*36:.1f} to {omega_dry[3].max()*36:.1f} hPa/h)")

## 6. Visualise ω at Multiple Levels

In [ ]:
levels_to_plot = [3, 5, 7]  # 500, 300, 200 hPa
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, kidx in zip(axes, levels_to_plot):
    plev = plevs_pa[kidx] / 100
    w = omega_dry[kidx] * 100  # Pa/s → hPa/s for readability
    vmax = np.nanmax(np.abs(w))
    if vmax < 1e-10:
        vmax = 1.0
    im = ax.contourf(lon, lat, w, levels=np.linspace(-vmax, vmax, 21),
                      cmap="RdBu")  # red = descent (positive ω)
    ax.set_title(f"ω at {plev:.0f} hPa [hPa/s]")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    plt.colorbar(im, ax=ax, shrink=0.8)

plt.suptitle("QG Vertical Velocity (red = descent, blue = ascent)",
             fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

## 7. Cross-Section

Longitude–pressure cross-section at 40°N (where the trough is located).

In [ ]:
j40 = np.argmin(np.abs(lat - 40))  # latitude index closest to 40°N
omega_xsec = omega_dry[:, j40, :] * 100  # hPa/s

fig, ax = plt.subplots(figsize=(14, 5))
vmax = np.nanmax(np.abs(omega_xsec))
if vmax < 1e-10:
    vmax = 1.0
im = ax.contourf(lon, plevs_pa / 100, omega_xsec,
                  levels=np.linspace(-vmax, vmax, 25), cmap="RdBu")
ax.invert_yaxis()
ax.set_xlabel("Longitude")
ax.set_ylabel("Pressure (hPa)")
ax.set_title(f"ω cross-section at {lat[j40]:.0f}°N (hPa/s)")
plt.colorbar(im, ax=ax, label="ω [hPa/s]")
plt.tight_layout()
plt.show()

## 8. Constant vs. Variable σ₀

Compare the omega field using constant static stability ($\sigma_0 = 2 \times 10^{-6}$)
vs. the profile-derived stability.

In [ ]:
omega_var_sigma = solve_qg_omega(
    ug, vg, t_3d, lat, lon, plevs_pa,
    use_constant_sigma=False,
)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for ax, omega, label in zip(axes,
                             [omega_dry, omega_var_sigma],
                             ["Constant σ₀", "Variable σ₀(p)"]):
    w = omega[3] * 100
    vmax = max(np.nanmax(np.abs(w)), 1e-10)
    im = ax.contourf(lon, lat, w, levels=np.linspace(-vmax, vmax, 21),
                      cmap="RdBu")
    ax.set_title(f"500 hPa ω — {label}")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    plt.colorbar(im, ax=ax, label="hPa/s")

plt.tight_layout()
plt.show()

---

**Summary**: The `solve_qg_omega` solver computes QG vertical motion from geostrophic wind and temperature,
showing ascent (blue) ahead of the trough and descent (red) behind — the classic Q-vector pattern.